<a href="https://colab.research.google.com/github/itsJonnie/Movie-Recommendation-System/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# INSTALL REQUIRED LIBRARIES:
# pip install pandas numpy scikit-learn

### Read data into notebook from ML-100K.zip (https://grouplens.org/datasets/movielens/100k/)

In [3]:
file_path = "/content/drive/My Drive/Colab Notebooks/Datasets//ml-100k.zip"

import os
if os.path.exists(file_path):
    print("File found!")
else:
    print("File not found. Check the path.")


File found!


In [9]:
import zipfile

file_path = "/content/drive/My Drive/Colab Notebooks/Datasets/ml-100k.zip"

with zipfile.ZipFile(file_path, "r") as zip_ref:
    zip_ref.extractall("/content")  # Extract to /content instead

import os
print("After extraction, /content contains:", os.listdir("/content"))



After extraction, /content contains: ['.config', 'drive', 'ml-100k', 'sample_data']


In [12]:
import pandas as pd
ratings_path = "/content/ml-100k/u.data"
movies_path = "/content/ml-100k/u.item"

print(os.listdir("/content/ml-100k"))

# The u.data file is tab-separated (\t) and does not have a header row, so we must provide column names
ratings = pd.read_csv(
    ratings_path,
    sep="\t",
    names=["userId", "movieId", "rating", "timestamp"]
)



['u.data', 'ua.test', 'u3.test', 'ml-100k', 'README', 'u5.base', 'u4.base', 'ub.base', 'u1.base', 'u.user', 'u.item', 'u.occupation', 'ua.base', 'ub.test', 'u3.base', 'allbut.pl', 'u5.test', 'mku.sh', 'u1.test', 'u4.test', 'u2.test', 'u.info', 'u2.base', 'u.genre']


In [13]:
movies = pd.read_csv(movies_path, sep="|", encoding="latin-1", names=[
    "movieId", "title", "release_date", "video_release_date", "IMDB_url",
    "unknown", "Action", "Adventure", "Animation", "Children", "Comedy", "Crime",
    "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery",
    "Romance", "Sci-Fi", "Thriller", "War", "Western"
], usecols=[0, 1])  # Only keeping movieId and title

# Merge ratings with movie titles
df = ratings.merge(movies, on="movieId")

# Display first few rows
df.head()


,userId,movieId,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,186,302,3,891717742,L.A. Confidential (1997)
2,22,377,1,878887116,Heavyweights (1994)
3,244,51,2,880606923,Legends of the Fall (1994)
4,166,346,1,886397596,Jackie Brown (1997)


In [17]:
# change timestamp to a String Format (e.g., "YYYY-MM-DD HH:MM:SS")

df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
df.head()


,userId,movieId,rating,timestamp,title,readable_time
0,196,242,3,1997-12-04 15:55:49,Kolya (1996),1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22,L.A. Confidential (1997),1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36,Heavyweights (1994),1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03,Legends of the Fall (1994),1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16,Jackie Brown (1997),1998-02-02 05:33:16


### Create a user-movie matrix where rows represent users, columns represent movies, & values are the user's rating for that movie

In [18]:
# Create the pivot table
user_movie_matrix = df.pivot_table(index="userId", columns="title", values="rating")

# Fill missing values with 0 (unrated movies become 0)
user_movie_matrix = user_movie_matrix.fillna(0) # 0 shows if a user has not left a rating for that movie

user_movie_matrix.head()


title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,2.0,5.0,0.0,0.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0


### Use K-Nearest Neighbors (k-NN) to Build a User-Based Collaborative Filtering Model


In [19]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Convert user-movie matrix to a NumPy array
user_movie_matrix_np = user_movie_matrix.to_numpy()

# Initialize the k-NN model
knn = NearestNeighbors(
    metric='cosine',  # measures similarity by cosine distance
    algorithm='brute' # straightforward search
)

# Train (fit) the model
knn.fit(user_movie_matrix_np)


NearestNeighbors(algorithm='brute', metric='cosine')

### Make a Recommendation Function


* Find neighbors using knn.kneighbors().
* Gather each neighbor’s top-rated movies.
* Exclude movies the current user has already seen.
* Return a few unseen recommendations

In [21]:
def recommend_movies(user_id, num_neighbors=5, num_recommendations=5):
    # Convert userId to matrix index (if userId starts at 1, index is userId - 1)
    user_index = user_id - 1

    # Get distances and neighbor indices for this user
    distances, indices = knn.kneighbors(
        [user_movie_matrix_np[user_index]],
        n_neighbors=num_neighbors+1  # +1 includes the user itself
    )

    # The first neighbor is the user itself, so skip it
    neighbor_indices = indices[0][1:]

    # We'll collect the top movies from each neighbor
    recommended_movies = []

    # Convert user_movie_matrix back to DataFrame for easier indexing
    user_movie_df = pd.DataFrame(user_movie_matrix_np,
                                 index=user_movie_matrix.index,
                                 columns=user_movie_matrix.columns)

    # For each similar user
    for neighbor_idx in neighbor_indices:
        # Get that user's ratings row
        neighbor_ratings = user_movie_df.iloc[neighbor_idx]

        # Sort by rating descending
        top_rated = neighbor_ratings.sort_values(ascending=False)

        # Take the top few movies
        recommended_movies.extend(top_rated.index[:num_recommendations])

    # Remove duplicates & keep only movies the user hasn't rated
    recommended_movies = list(set(recommended_movies))

    # Filter out movies the current user has already seen
    current_user_ratings = user_movie_df.iloc[user_index]
    seen_movies = current_user_ratings[current_user_ratings > 0].index
    unseen_recs = [m for m in recommended_movies if m not in seen_movies]

    # Return a few recommendations
    return unseen_recs[:num_recommendations]


### Test Recommendation Function

In [22]:
user_id = 10 # Pick a user (e.g., user_id = 10) and see what gets recommended:
recs = recommend_movies(user_id, num_neighbors=5, num_recommendations=5)
print(f"Recommendations for user {user_id}:\n", recs)


Recommendations for user 10:
 ['Dumbo (1941)', 'Princess Bride, The (1987)', 'Heathers (1989)', 'Boys, Les (1997)', 'Touch of Evil (1958)']


In [28]:
# select all rows where userid = 10 since I want to see what movies they have rated

# Assuming 'df' is your merged DataFrame (ratings and movies)
user_10_df = df[df["userId"] == 10]
print(len(user_10_df))
user_10_df.head()

184


,userId,movieId,rating,timestamp,title,readable_time
40,10,16,4,1997-10-26 18:01:17,French Twist (Gazon maudit) (1995),1997-10-26 18:01:17
158,10,486,4,1997-10-26 17:27:26,Sabrina (1954),1997-10-26 17:27:26
386,10,175,3,1997-10-26 17:57:57,Brazil (1985),1997-10-26 17:57:57
544,10,611,5,1997-10-26 17:25:22,Laura (1944),1997-10-26 17:25:22
606,10,7,4,1997-10-26 18:56:50,Twelve Monkeys (1995),1997-10-26 18:56:50
